In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.neighbors import NearestNeighbors
import pandas as pd 
import numpy as np
import warnings
import time
import seaborn as sns
import matplotlib.pyplot as plt
import random
import nltk
from nltk.corpus import stopwords

warnings.filterwarnings('ignore')


In [2]:
df = pd.read_csv('dataset.csv')
df

,Meal_Id,Name,catagory,description,Veg_Non,Nutrient,Disease,Diet,Price
0,meal_id1,summer squash salad,salad,"white balsamic vinegar, lemon juice, lemon rin...",veg,fiber,obesity diabeties hypertension goitre,alkaline_diet low_fat_diet ketogenic_diet low...,485
1,meal_id2,chicken minced salad,salad,"olive oil, chicken mince, garlic (minced), oni...",non-veg,fiber,anemia pregnancy hypertension rickets goitre ...,low_fat_diet low_carb_diet ketogenic_diet low...,600
2,meal_id3,sweet chilli almonds,chilli,"almonds whole, egg white, curry leaves, salt, ...",veg,vitamin_a,hypertension scurvy heart_disease goitre kidn...,alkaline_diet low_fat_diet paleo_diet Mediter...,255
3,meal_id4,tricolour salad,salad,"vinegar, honey/sugar, soy sauce, salt, garlic ...",veg,fiber,obesity goitre hypertension,low_fat_diet ketogenic_diet low_sodium_diet h...,615
4,meal_id5,gluten-free christmas cake,cake,"christmas dry fruits (pre-soaked), orange zest...",veg,vitamin_a,goitre kidney_disease,high_protien_diet,465
...,...,...,...,...,...,...,...,...,...
507,meal_id305,sunga pork,curry,curry,veg,iron,pregnancy goitre,low_fat_diet paleo_diet vegan_diet high_fiber...,605
508,meal_id306,banana chips,banana,r dried slices of bananas (fruits of herbaceou...,veg,magnesium,hypertension,alkaline_diet high_protien_diet vegan_diet hi...,295
509,meal_id307,bhurji- egg,egg,"made using indian spices, onion, tomatoes, gre...",non-veg,vitamin_a,hypertension rickets goitre kidney_disease pr...,low_sodium_diet high_protien_diet low_fat_die...,645
510,meal_id308,flattened rice / pohe,rice,"ible, dehusked rice which is flattened into fl...",veg,carbohydrates,anemia cancer scurvy heart_disease eye_diseas...,alkaline_diet low_fat_diet ketogenic_diet veg...,525


In [3]:
nutrient_dummies = df.Nutrient.str.get_dummies()
disease_dummies = df.Disease.str.get_dummies(sep=' ')
diet_dummies = df.Diet.str.get_dummies(sep=' ')
feature_df = pd.concat([nutrient_dummies,disease_dummies,diet_dummies],axis=1)



In [4]:
feature_df

,calcium,carbohydrates,chloride,fiber,iodine,iron,magnesium,manganese,phosphorus,potassium,...,hormone_diet,ketogenic_diet,low_carb_diet,low_fat_diet,low_sodium_diet,omni_diet,paleo_diet,type_a_diet,type_o_diet,vegan_diet
0,0,0,0,1,0,0,0,0,0,0,...,0,1,0,1,1,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,1,1,1,1,1,0,0,1,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,1,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,1,0,1,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
508,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
509,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,0,0,0,0,0
510,0,1,0,0,0,0,0,0,0,0,...,1,1,0,1,0,0,0,0,0,1


In [5]:
total_features = feature_df.columns

In [6]:
total_features

Index(['calcium', 'carbohydrates', 'chloride', 'fiber', 'iodine', 'iron',
       'magnesium', 'manganese', 'phosphorus', 'potassium', 'protien',
       'selenium', 'sodium', 'vitamin_a', 'vitamin_c', 'vitamin_d',
       'vitamin_e', 'anemia', 'cancer', 'diabeties', 'eye_disease', 'goitre',
       'heart_disease', 'hypertension', 'kidney_disease', 'obesity',
       'pregnancy', 'rickets', 'scurvy', 'Mediterranean_diet', 'alkaline_diet',
       'dash_diet', 'gluten_free_diet', 'high_fiber_diet', 'high_protien_diet',
       'hormone_diet', 'ketogenic_diet', 'low_carb_diet', 'low_fat_diet',
       'low_sodium_diet', 'omni_diet', 'paleo_diet', 'type_a_diet',
       'type_o_diet', 'vegan_diet'],
      dtype='object')

In [7]:
d = dict()
for i in total_features:
    d[i]= 0
print(d)

{'calcium': 0, 'carbohydrates': 0, 'chloride': 0, 'fiber': 0, 'iodine': 0, 'iron': 0, 'magnesium': 0, 'manganese': 0, 'phosphorus': 0, 'potassium': 0, 'protien': 0, 'selenium': 0, 'sodium': 0, 'vitamin_a': 0, 'vitamin_c': 0, 'vitamin_d': 0, 'vitamin_e': 0, 'anemia': 0, 'cancer': 0, 'diabeties': 0, 'eye_disease': 0, 'goitre': 0, 'heart_disease': 0, 'hypertension': 0, 'kidney_disease': 0, 'obesity': 0, 'pregnancy': 0, 'rickets': 0, 'scurvy': 0, 'Mediterranean_diet': 0, 'alkaline_diet': 0, 'dash_diet': 0, 'gluten_free_diet': 0, 'high_fiber_diet': 0, 'high_protien_diet': 0, 'hormone_diet': 0, 'ketogenic_diet': 0, 'low_carb_diet': 0, 'low_fat_diet': 0, 'low_sodium_diet': 0, 'omni_diet': 0, 'paleo_diet': 0, 'type_a_diet': 0, 'type_o_diet': 0, 'vegan_diet': 0}


In [8]:
sample_input = ['high_protien_diet','gluten_free_diet','diabeties','anemia','calcium','protien']

for i in sample_input:
    
    d[i] = 1

final_input = list(d.values())

In [9]:

        

model = NearestNeighbors(n_neighbors=40,algorithm='ball_tree')


model.fit(feature_df)

df_results = pd.DataFrame(columns=list(df.columns))



distnaces , indices = model.kneighbors([final_input])

for i in list(indices):
    df_results = df_results.append(df.loc[i])

df_results = df_results.filter(['Name','Nutrient','Veg_Non','Price','Review','Diet','Disease','description'])
df_results = df_results.drop_duplicates(subset=['Name'])
df_results = df_results.reset_index(drop=True)


In [10]:
df_results

,Name,Nutrient,Veg_Non,Price,Diet,Disease,description
0,corn pulao,protien,veg,410,high_protien_diet low_fat_diet gluten_free_diet,diabeties goitre,"बासमती चावल, अमेरिकन कॉर्न के दाने, जैतून का त..."
1,veg summer rolls,carbohydrates,veg,545,high_protien_diet,diabeties,"rice paper sheets, iceberg lettuce, carrot, be..."
2,oats &amp; shallots pulao,protien,veg,470,high_protien_diet dash_diet gluten_free_diet,hypertension,"चावल, हरा धनिया, हरी मिर्च, प्याज , दालचीनी, इ..."
3,oats khichdi,protien,veg,690,high_protien_diet dash_diet gluten_free_diet,hypertension,"क्विक कुकिंग ओट्स, मूंगदाल, जीरा, हल्दी पाउडर,..."
4,almond pearls,protien,veg,550,gluten_free_diet,hypertension,"toasted almonds, blueberries, oats, corn flake..."
5,apple kheer,vitamin_e,veg,355,high_protien_diet,obesity diabeties,"apples, basmati rice, nuscovado sugar (you can..."
6,cajun spiced turkey wrapped with bacon,vitamin_c,non-veg,485,high_protien_diet,cancer,"turkey breast, cajun spice, spinach leaves (co..."
7,cinnamon star cookies,protien,veg,530,ketogenic_diet,cancer diabeties hypertension,"मक्खन, कैस्टर शुगर, क्रिसमस मिक्स स्पाइसी, दाल..."
8,apple rabdi,calcium,veg,335,high_protien_diet vegan_diet low_fat_diet ket...,obesity diabeties,"apples, milk, sugar, green cardamoms, almonds ..."
9,lemon honey glazed sous vide corn on the cob,protien,non-veg,310,low_fat_diet gluten_free_diet high_fiber_diet...,diabeties goitre scurvy,"young corn on the cob, honey, lemon juice, gar..."


In [11]:
df_results['Disease']

0                   diabeties goitre
1                          diabeties
2                       hypertension
3                       hypertension
4                       hypertension
5                  obesity diabeties
6                             cancer
7      cancer diabeties hypertension
8                  obesity diabeties
9            diabeties goitre scurvy
10                            goitre
11      obesity rickets hypertension
12                      hypertension
13          obesity diabeties anemia
14                      hypertension
15                  diabeties goitre
16                     anemia goitre
17                            scurvy
18          obesity diabeties anemia
19          obesity diabeties anemia
20             goitre kidney_disease
21          obesity diabeties anemia
22                      hypertension
23                           obesity
Name: Disease, dtype: object

In [12]:
df_results[df_results['Price'] >500]

,Name,Nutrient,Veg_Non,Price,Diet,Disease,description
1,veg summer rolls,carbohydrates,veg,545,high_protien_diet,diabeties,"rice paper sheets, iceberg lettuce, carrot, be..."
3,oats khichdi,protien,veg,690,high_protien_diet dash_diet gluten_free_diet,hypertension,"क्विक कुकिंग ओट्स, मूंगदाल, जीरा, हल्दी पाउडर,..."
4,almond pearls,protien,veg,550,gluten_free_diet,hypertension,"toasted almonds, blueberries, oats, corn flake..."
7,cinnamon star cookies,protien,veg,530,ketogenic_diet,cancer diabeties hypertension,"मक्खन, कैस्टर शुगर, क्रिसमस मिक्स स्पाइसी, दाल..."
12,almond and amaranth ladoo,calcium,veg,570,low_fat_diet ketogenic_diet gluten_free_diet ...,hypertension,"popped amaranth seeds, jaggery, almonds (slive..."
14,ragi oats ladoo (laddu),vitamin_e,veg,550,high_protien_diet dash_diet gluten_free_diet,hypertension,"ragi flour, oats flour, dates (ripe), milk, ho..."
16,shrimp &amp; cilantro ceviche,vitamin_a,veg,685,ketogenic_diet,anemia goitre,"prawns, gherkin, onion, cilantro, mix bell pep..."
22,cashew nut cookies,protien,veg,655,high_protien_diet vegan_diet dash_diet ketoge...,hypertension,"cashew paste, ghee, khaand (a sweetening agent..."
23,buldak (hot and spicy chicken),magnesium,non-veg,545,high_protien_diet high_fiber_diet,obesity,"chicken drum sticks/ chicken breast, soy sauce..."


In [13]:

df_results[df_results['Disease'].str.contains("obesity")]

,Name,Nutrient,Veg_Non,Price,Diet,Disease,description
5,apple kheer,vitamin_e,veg,355,high_protien_diet,obesity diabeties,"apples, basmati rice, nuscovado sugar (you can..."
8,apple rabdi,calcium,veg,335,high_protien_diet vegan_diet low_fat_diet ket...,obesity diabeties,"apples, milk, sugar, green cardamoms, almonds ..."
11,californian breakfast benedict,protien,veg,290,high_protien_diet,obesity rickets hypertension,"brioche loaf, avocado paste, eggs, tomato, spi..."
13,rice kheer,calcium,veg,300,low_fat_diet ketogenic_diet vegan_diet high_p...,obesity diabeties anemia,"milk, rice (washed), sugar, raisins, green car..."
18,rice,carbohydrates,veg,350,high_protien_diet vegan_diet dash_diet ketoge...,obesity diabeties anemia,NaN
19,puffed rice,carbohydrates,veg,265,high_protien_diet vegan_diet dash_diet ketoge...,obesity diabeties anemia,grain made from rice; usually made by heating ...
21,puffed rice squares,carbohydrates,veg,375,high_protien_diet vegan_diet dash_diet ketoge...,obesity diabeties anemia,"puffed rice, nuts, honey, jaggery, butter, kew..."
23,buldak (hot and spicy chicken),magnesium,non-veg,545,high_protien_diet high_fiber_diet,obesity,"chicken drum sticks/ chicken breast, soy sauce..."


In [14]:
my_df_check=pd.read_csv('dataset.csv')
my_df_check.head()

,Meal_Id,Name,catagory,description,Veg_Non,Nutrient,Disease,Diet,Price
0,meal_id1,summer squash salad,salad,"white balsamic vinegar, lemon juice, lemon rin...",veg,fiber,obesity diabeties hypertension goitre,alkaline_diet low_fat_diet ketogenic_diet low...,485
1,meal_id2,chicken minced salad,salad,"olive oil, chicken mince, garlic (minced), oni...",non-veg,fiber,anemia pregnancy hypertension rickets goitre ...,low_fat_diet low_carb_diet ketogenic_diet low...,600
2,meal_id3,sweet chilli almonds,chilli,"almonds whole, egg white, curry leaves, salt, ...",veg,vitamin_a,hypertension scurvy heart_disease goitre kidn...,alkaline_diet low_fat_diet paleo_diet Mediter...,255
3,meal_id4,tricolour salad,salad,"vinegar, honey/sugar, soy sauce, salt, garlic ...",veg,fiber,obesity goitre hypertension,low_fat_diet ketogenic_diet low_sodium_diet h...,615
4,meal_id5,gluten-free christmas cake,cake,"christmas dry fruits (pre-soaked), orange zest...",veg,vitamin_a,goitre kidney_disease,high_protien_diet,465


In [15]:
my_df_check.Diet.unique()

array([' alkaline_diet low_fat_diet ketogenic_diet low_sodium_diet high_fiber_diet high_protien_diet dash_diet',
       ' low_fat_diet low_carb_diet ketogenic_diet low_sodium_diet vegan_diet high_fiber_diet hormone_diet high_protien_diet type_a_diet',
       ' alkaline_diet low_fat_diet paleo_diet Mediterranean_diet low_sodium_diet high_fiber_diet high_protien_diet dash_diet',
       ' low_fat_diet ketogenic_diet low_sodium_diet high_fiber_diet high_protien_diet',
       ' high_protien_diet',
       ' high_protien_diet vegan_diet low_fat_diet ketogenic_diet',
       ' high_protien_diet high_fiber_diet ketogenic_diet',
       ' alkaline_diet',
       ' alkaline_diet low_fat_diet ketogenic_diet gluten_free_diet vegan_diet high_protien_diet',
       ' alkaline_diet ketogenic_diet gluten_free_diet vegan_diet high_protien_diet dash_diet',
       ' alkaline_diet low_fat_diet paleo_diet ketogenic_diet low_sodium_diet gluten_free_diet vegan_diet high_fiber_diet high_protien_diet dash_diet',
  

In [16]:
my_df_check.Nutrient.unique()

array(['fiber', 'vitamin_a', 'calcium', 'magnesium', 'sodium',
       'vitamin_c', 'protien', 'vitamin_e', 'iron', 'selenium',
       'carbohydrates', 'chloride', 'potassium', 'vitamin_d', 'manganese',
       'phosphorus', 'iodine'], dtype=object)

In [17]:
my_df_check.Disease.unique()

array([' obesity diabeties hypertension goitre',
       ' anemia pregnancy hypertension rickets goitre kidney_disease obesity',
       ' hypertension scurvy heart_disease goitre kidney_disease pregnancy',
       ' obesity goitre hypertension', ' goitre kidney_disease',
       ' goitre', ' hypertension heart_disease',
       ' pregnancy scurvy goitre kidney_disease obesity',
       ' obesity cancer hypertension',
       ' pregnancy hypertension scurvy obesity diabeties',
       ' pregnancy hypertension scurvy goitre obesity',
       ' diabeties goitre scurvy', ' scurvy',
       ' anemia cancer heart_disease kidney_disease obesity diabeties',
       ' hypertension',
       ' anemia hypertension cancer scurvy heart_disease kidney_disease diabeties',
       ' goitre hypertension kidney_disease', ' goitre hypertension',
       ' diabeties goitre', ' obesity diabeties',
       ' anemia hypertension scurvy heart_disease goitre kidney_disease pregnancy diabeties',
       ' obesity scurvy hyper

In [18]:
my_df_check.Diet.unique()

array([' alkaline_diet low_fat_diet ketogenic_diet low_sodium_diet high_fiber_diet high_protien_diet dash_diet',
       ' low_fat_diet low_carb_diet ketogenic_diet low_sodium_diet vegan_diet high_fiber_diet hormone_diet high_protien_diet type_a_diet',
       ' alkaline_diet low_fat_diet paleo_diet Mediterranean_diet low_sodium_diet high_fiber_diet high_protien_diet dash_diet',
       ' low_fat_diet ketogenic_diet low_sodium_diet high_fiber_diet high_protien_diet',
       ' high_protien_diet',
       ' high_protien_diet vegan_diet low_fat_diet ketogenic_diet',
       ' high_protien_diet high_fiber_diet ketogenic_diet',
       ' alkaline_diet',
       ' alkaline_diet low_fat_diet ketogenic_diet gluten_free_diet vegan_diet high_protien_diet',
       ' alkaline_diet ketogenic_diet gluten_free_diet vegan_diet high_protien_diet dash_diet',
       ' alkaline_diet low_fat_diet paleo_diet ketogenic_diet low_sodium_diet gluten_free_diet vegan_diet high_fiber_diet high_protien_diet dash_diet',
  

In [19]:
class Profile:
    
    df = pd.read_csv('dataset.csv') # static variable
    
    def __init__(self,diet,disease,Nutrient,food_type,favorite_food):
        self.diet = diet
        self.disease = disease
        self.nutrient = Nutrient
        self.type = food_type
        self.like = favorite_food
        self.df2 = pd.DataFrame(columns=list(Profile.df.columns))
        self.df3 = pd.DataFrame(columns=list(Profile.df.columns))
        self.df4 = pd.DataFrame(columns=list(Profile.df.columns))
        self.df5 = pd.DataFrame(columns=list(Profile.df.columns))
        self.df6 = pd.DataFrame(columns=list(Profile.df.columns))
        
    def removestop(self,tokens):
        stop = set(stopwords.words('english'))
        file = open('stopwords.txt','r')
        l = list(file.read().split())
        stop = list(stop) +l
        l = [token for token in tokens if token not in stop]
        return l
            
    def inputs(self,diet,disease,Nutrient,food_type,favorite_food):
        
        if Nutrient:
            self.df2 = Profile.df[Profile.df.Nutrient.isin(Nutrient)]
            self.df2 = self.df2.reset_index()
       
        if food_type:
            self.df2 = self.df2[self.df2.Veg_Non.isin(food_type)]
            self.df2 = self.df2.reset_index()
      
        if diet:
            for i in range(self.df2.shape[0]):
                l = str(self.df2.loc[i,'Diet']).split()
                
                for d in diet:
                    if d in l:
                        self.df4=self.df4.append(self.df2.loc[i])
 
        if disease:
            for i in range(self.df2.shape[0]):
                l = str(self.df2.loc[i,'Disease']).split()
                for d in disease:
                    if d in l:
                        self.df5=self.df5.append(self.df2.loc[i])

        if favorite_food:
            f = self.removestop(favorite_food.split())
            for i in range(Profile.df.shape[0]):
                n = [j.lower() for j in str(Profile.df.loc[i,'Name']).split()]
                for j in n:
                    for k in f:
                        if k==j:
                            self.df6=self.df6.append(Profile.df.loc[i])
            for i in range(Profile.df.shape[0]):
                n = [j.lower() for j in str(Profile.df.loc[i,'description']).split()]
                for j in n:
                    for k in f:
                        if k==j:
                            self.df6=self.df6.append(df.loc[i])
            for i in range(Profile.df.shape[0]):
                n = [j.lower() for j in str(Profile.df.loc[i,'catagory']).split()]
                for j in n:
                    for k in f:
                        if k==j:
                            self.df6=self.df6.append(Profile.df.loc[i])
            
        return self.df2,self.df3,self.df4,self.df5,self.df6
    
    def get_profile(self):
        df2,df3,df4,df5,df6 = self.inputs(self.diet,self.disease,self.nutrient,self.type,self.like)
        
        df_merge = pd.concat([df2,df3,df4,df5,df6],axis=0).drop_duplicates(subset='Name')
        df_merge = df_merge.filter(['Name','Nutrient','Veg_Non','Price','Review','description'])
        print(df_merge.shape)
        
        return df_merge

In [20]:
ob = Profile(['low_sodium_diet','low_fat_diet'],['diabetes','rickets'],['calcium','vitamin_c'],['non-veg'],'i love chineese')

profile = ob.get_profile()
profile

(12, 5)


,Name,Nutrient,Veg_Non,Price,description
0,almond and chicken momos (without shell),calcium,non-veg,415,"chicken mince, garlic, carrots, spring onion, ..."
1,chicken parmigiana with tomato sauce,vitamin_c,non-veg,485,"for chicken parmigiana:, chicken breast, egg w..."
2,sous-vide salmon tikka,calcium,non-veg,465,"norwegian salmon, black garlic pickle, butter,..."
4,riceless chicken biryani,vitamin_c,non-veg,360,"malabar peppercorn, cinnamon stick, green card..."
5,chicken roulade,calcium,non-veg,665,"chicken breasts, olives, jalapenos, bell peppe..."
6,cheese chicken kebabs,calcium,non-veg,270,"चिकन थाईज़, लहसुन का पेस्ट, लहसुन का पेस्ट, पी..."
7,cajun spiced turkey wrapped with bacon,vitamin_c,non-veg,555,"turkey breast, cajun spice, spinach leaves (co..."
8,seared salmon in tabasco butter,calcium,non-veg,265,"butter, tabasco, chives, salt, salmon fillet, ..."
9,"risotto lobster with parmesan egg pancake, con...",vitamin_c,non-veg,455,"lobster shell, carrot, leeks, garlic cloves, t..."
10,fish with jamun sauce,vitamin_c,non-veg,560,"jamun, sugar, chilli, garlic cloves (minced), ..."


In [21]:
ob = Profile(['low_sodium_diet','low_fat_diet'],['diabetes','rickets'],['calcium','vitamin_c'],['veg'],'i love chineese')

profile = ob.get_profile()
profile

(38, 5)


,Name,Nutrient,Veg_Non,Price,description
0,japanese curry arancini with barley salsa,calcium,veg,630,"japanese curry, sticky rice, cheese inside ric..."
1,cream of almond soup,calcium,veg,290,"vegetable stock, skimmed milk, toasted almonds..."
2,broccoli and almond soup,vitamin_c,veg,515,"vegetable stock, broccoli, ground almonds (toa..."
3,baked namakpara with roasted almond dip,calcium,veg,425,"almonds (crushed), tomato, garlic cloves, basi..."
4,grilled almond barfi (sugar free),calcium,veg,645,"khoya, sweetener (optional), almonds (crushed)"
5,apple rabdi,calcium,veg,335,"apples, milk, sugar, green cardamoms, almonds ..."
6,green lentil dessert fudge,calcium,veg,210,"whole moong beans, cow ghee, raisins, whole mi..."
7,spinach and feta crepes,vitamin_c,veg,245,"milk (whole fat or skimmed), flour, water, but..."
8,baked almond kofta,calcium,veg,610,"potato (boiled), nutmeg, milk, almonds (crushe..."
9,almond and amaranth ladoo,calcium,veg,685,"popped amaranth seeds, jaggery, almonds (slive..."


In [22]:
ob = Profile(['low_sodium_diet','low_fat_diet'],['diabetes','rickets'],['calcium','vitamin_c'],['veg'],'i love indian food')

profile = ob.get_profile()
profile

(42, 5)


,Name,Nutrient,Veg_Non,Price,description
0,japanese curry arancini with barley salsa,calcium,veg,630,"japanese curry, sticky rice, cheese inside ric..."
1,cream of almond soup,calcium,veg,290,"vegetable stock, skimmed milk, toasted almonds..."
2,broccoli and almond soup,vitamin_c,veg,515,"vegetable stock, broccoli, ground almonds (toa..."
3,baked namakpara with roasted almond dip,calcium,veg,425,"almonds (crushed), tomato, garlic cloves, basi..."
4,grilled almond barfi (sugar free),calcium,veg,645,"khoya, sweetener (optional), almonds (crushed)"
5,apple rabdi,calcium,veg,335,"apples, milk, sugar, green cardamoms, almonds ..."
6,green lentil dessert fudge,calcium,veg,210,"whole moong beans, cow ghee, raisins, whole mi..."
7,spinach and feta crepes,vitamin_c,veg,245,"milk (whole fat or skimmed), flour, water, but..."
8,baked almond kofta,calcium,veg,610,"potato (boiled), nutmeg, milk, almonds (crushe..."
9,almond and amaranth ladoo,calcium,veg,685,"popped amaranth seeds, jaggery, almonds (slive..."


In [23]:
class Profile1:
    
    df = pd.read_csv('dataset.csv') # static variable
    
    def __init__(self,diet,disease,Nutrient,food_type):
        self.diet = diet
        self.disease = disease
        self.nutrient = Nutrient
        self.type = food_type
       
        self.df2 = pd.DataFrame(columns=list(Profile.df.columns))
        self.df3 = pd.DataFrame(columns=list(Profile.df.columns))
        self.df4 = pd.DataFrame(columns=list(Profile.df.columns))
        self.df5 = pd.DataFrame(columns=list(Profile.df.columns))
        
        
    def removestop(self,tokens):
        stop = set(stopwords.words('english'))
        file = open('stopwords.txt','r')
        l = list(file.read().split())
        stop = list(stop) +l
        l = [token for token in tokens if token not in stop]
        return l
            
    def inputs(self,diet,disease,Nutrient,food_type):
        
        if Nutrient:
            self.df2 = Profile.df[Profile.df.Nutrient.isin(Nutrient)]
            self.df2 = self.df2.reset_index()
       
        if food_type:
            self.df2 = self.df2[self.df2.Veg_Non.isin(food_type)]
            self.df2 = self.df2.reset_index()
      
        if diet:
            for i in range(self.df2.shape[0]):
                l = str(self.df2.loc[i,'Diet']).split()
                
                for d in diet:
                    if d in l:
                        self.df4=self.df4.append(self.df2.loc[i])
 
        if disease:
            for i in range(self.df2.shape[0]):
                l = str(self.df2.loc[i,'Disease']).split()
                for d in disease:
                    if d in l:
                        self.df5=self.df5.append(self.df2.loc[i])

        
            
        return self.df2,self.df3,self.df4,self.df5
    
    def get_profile(self):
        df2,df3,df4,df5 = self.inputs(self.diet,self.disease,self.nutrient,self.type)
        
        df_merge = pd.concat([df2,df3,df4,df5],axis=0).drop_duplicates(subset='Name')
        df_merge = df_merge.filter(['Name','Nutrient','Veg_Non','Price','Review','description'])
        print(df_merge.shape)
        
        return df_merge

In [24]:
ob = Profile1(['low_sodium_diet','low_fat_diet'],['diabetes','rickets'],['calcium','vitamin_c'],['veg'])

profile = ob.get_profile()
profile

(38, 5)


,Name,Nutrient,Veg_Non,Price,description
0,japanese curry arancini with barley salsa,calcium,veg,630,"japanese curry, sticky rice, cheese inside ric..."
1,cream of almond soup,calcium,veg,290,"vegetable stock, skimmed milk, toasted almonds..."
2,broccoli and almond soup,vitamin_c,veg,515,"vegetable stock, broccoli, ground almonds (toa..."
3,baked namakpara with roasted almond dip,calcium,veg,425,"almonds (crushed), tomato, garlic cloves, basi..."
4,grilled almond barfi (sugar free),calcium,veg,645,"khoya, sweetener (optional), almonds (crushed)"
5,apple rabdi,calcium,veg,335,"apples, milk, sugar, green cardamoms, almonds ..."
6,green lentil dessert fudge,calcium,veg,210,"whole moong beans, cow ghee, raisins, whole mi..."
7,spinach and feta crepes,vitamin_c,veg,245,"milk (whole fat or skimmed), flour, water, but..."
8,baked almond kofta,calcium,veg,610,"potato (boiled), nutmeg, milk, almonds (crushe..."
9,almond and amaranth ladoo,calcium,veg,685,"popped amaranth seeds, jaggery, almonds (slive..."


In [25]:
ob = Profile1(['low_sodium_diet'],['diabetes'],['vitamin_c'],['veg'])

profile = ob.get_profile()
profile

(11, 5)


,Name,Nutrient,Veg_Non,Price,description
0,broccoli and almond soup,vitamin_c,veg,515,"vegetable stock, broccoli, ground almonds (toa..."
1,spinach and feta crepes,vitamin_c,veg,245,"milk (whole fat or skimmed), flour, water, but..."
3,grilled lemon margarita,vitamin_c,veg,410,"vanilla infused tequila, vanilla liqueur, lemo..."
4,spanish artichoke and spinach dip,vitamin_c,veg,425,"spinach, onion, cream, garlic, nutmeg, salt, l..."
6,red wine braised mushroom flatbread,vitamin_c,veg,580,"olive oil, fresh buffalo mozzarella cheese, ca..."
7,bread with tomatoes and olives,vitamin_c,veg,435,"फ्रेंच ब्रेड लोफ, टोमैटो, एक्स्ट्रा वर्जिन ऑलि..."
8,duo of chocolate and strawberry,vitamin_c,veg,220,"dark chocolate, white chocolate, strawberries"
9,broccoli souffle,vitamin_c,veg,610,"broccoli, butter, extra virgin olive oil, all-..."
11,banana phirni tartlets with fresh strawberries,vitamin_c,veg,570,"basmati rice (soaked in water), milk, cardamom..."
15,strawberry quinoa pancakes,vitamin_c,veg,210,"quinoa, milk, olive oil, egg (slightly beaten)..."


In [26]:

l_name=[]
l_Nutrient=[]
l_Veg_Non=[]
l_description=[]
for i in range(len(profile)):
    l_name.append(profile.iloc[i][0])
    l_Nutrient.append(profile.iloc[i][1])
    l_Veg_Non.append(profile.iloc[i][2])
    l_description.append(profile.iloc[i][3])
print(l_name)

['broccoli and almond soup', 'spinach and feta crepes', 'grilled lemon margarita', 'spanish artichoke and spinach dip', 'red wine braised mushroom flatbread', 'bread with tomatoes and olives', 'duo of chocolate and strawberry', 'broccoli souffle', 'banana phirni tartlets with fresh strawberries', 'strawberry quinoa pancakes', 'spinach &amp; banana pancakes']


In [ ]:
while True:
    f=open('task.txt','r')
    task=f.read()
    f.close()
    if task=='recommend':
        f=open('input.txt','r')
        inp=f.read()
        f.close()
        l=inp.split('#')
        ob = Profile1([l[0]],[l[1]],[l[2]],[l[3]])
        profile = ob.get_profile()
        l_name=[]
        l_Nutrient=[]
        l_Veg_Non=[]
        l_description=[]
        for i in range(len(profile)):
            l_name.append(profile.iloc[i][0])
            l_Nutrient.append(profile.iloc[i][1])
            l_Veg_Non.append(profile.iloc[i][2])
            l_description.append(profile.iloc[i][4])
        print(l_name)
        print(l_Nutrient)
        print(l_Veg_Non)
        print(l_description)
        names=''
        nutrient=''
        veg_nonveg=''
        description=''
        for i in l_name:
            names=i+'#'+names
        for i in l_Nutrient:
            nutrient=i+'#'+nutrient   
        for i in l_Veg_Non:
            veg_nonveg=i+'#'+veg_nonveg 
        for i in l_description:
            description=i+'#'+description
        f=open('names.txt','w')
        f.write(names)
        f.close()
        f=open('nutrient.txt','w')
        f.write(nutrient)
        f.close()
        f=open('veg_nonveg.txt','w')
        f.write(veg_nonveg)
        f.close()
        f=open('description.txt','w')
        f.write(description)
        f.close()
        f=open('task.txt','w')
        f.write('')
        f.close()
        
        